In [1]:
# Import required libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas

import numpy as np
from scipy.stats import linregress

import matplotlib.pyplot as plt
import hvplot.pandas

In [3]:
### import and read csv file

path = 'Output/full_indicators_by_country.csv'
full_indicators_countries_df = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

full_indicators_countries_df.head(5)

,index,country,iso3Code,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,0,Afghanistan,AFG,32.400000,91.555556,58.432353,2.681272,327.0,36.628111,0.000000,...,0.012778,0.270084,3.220058,10.011889,7.579333,9.741222,11.616111,20.0,3.294,218.749241
1,1,Albania,ALB,81.455556,99.977778,42.382186,0.614119,1485.0,10.610444,78.238375,...,4.822222,5.576404,17.127848,13.158667,10.648444,14.884000,12.600667,37.0,1.809,197.990096
2,2,Algeria,DZA,99.677778,99.655556,17.353871,2.628728,89.0,23.088556,0.000000,...,0.574444,0.162233,0.736472,13.805556,8.501111,8.757778,12.170889,36.0,2.110,94.177289
3,3,American Samoa,ASM,0.000000,0.000000,14.150000,0.000000,0.0,11.422222,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000
4,4,Andorra,AND,100.000000,100.000000,39.983452,0.000000,0.0,6.466667,0.000000,...,10.430000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000


In [4]:
# this is to test the list that will be used in the fnction below

indicators_list = full_indicators_countries_df.columns.to_list()
indicators_list = indicators_list[2:len(indicators_list)-1]
indicators_list

['iso3Code',
 'clean fuels for cooking (% of population)',
 'Access to electricity (% of population)',
 'Agricultural land (% )',
 'Armed forces (% labor force)',
 'precipitation (mm per year)',
 'Birth rate, crude (per 1,000 people)',
 'Central government debt, total (% of GDP)',
 'Consumer price index (2010 = 100)',
 'Control of Corruption: Estimate',
 'Cost of business start-up (% of GNI per capita)',
 'Current account balance (% of GDP)',
 'Current health expenditure (% of GDP)',
 'Death rate, crude (per 1,000 people)',
 'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
 'doing business score',
 'Fertility rate, total (births per woman)',
 '% Firms using banks',
 'GDP growth (annual %)',
 'GDP per capita (constant 2015 US$)',
 'GDP per capita, PPP (constant 2021 international $)',
 'GNI per capita (2015 US$)',
 'government  consumption exp (% of GDP)',
 'Government expenditure on education, total (% of GDP)',
 'Human Capital Index',
 'Industry (

In [5]:
# function that compares all the indicators against ONE (immigration flow or immigration_100K) and generate a DF with al the regression parameters

# arguments are 
        ## the dataframe name
        ## number of index columns to be exculded

def select_indicator(dataframe,col_index):
    
    # create list of indicator from the Dataframe consolidated
    indicators_list = dataframe.columns.to_list()
    indicators_list = indicators_list[col_index:len(indicators_list)-1]
    indicators_list

    # create list of returned values
    indic_selection = []

    # loop through the indicators
    for ind in indicators_list :
        col1 = 'immigration_100k'
        col2 = ind


        df_regression = dataframe[[col1,col2]]
        
        #drop empty values

        df_regression = df_regression.dropna(how= 'any')


        # identify outliers
        var = df_regression[col2]
        q1 = np.quantile(var, 0.25) 
        q3 = np.quantile(var, 0.75)   
        iqr = q3 - q1
        outliers = var[(var > (q3 + (iqr * 1.5))) | (var < (q1 - (iqr * 1.5)))].tolist()

        # Exclude  outliers
        x = ~df_regression[col2].isin(outliers)
        dataframe_filtered = df_regression[x]

        x_axis = dataframe_filtered[col1]
        y_axis = dataframe_filtered[col2]

        ## calculate slope and intercept of the regression equation

        (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
        

        # append regresson values in list of dictionaries
        indic_selection.append({
                            'indicator A' : col1,
                            'indicator B' : col2,
                            'slope' : slope,
                            'intercept' :  intercept, 
                            'rvalue' : rvalue, 
                            'pvalue' : pvalue, 
                            'std' : stderr,
                            "rvalue_abs" : abs(rvalue)

                            })
    
    # create DF from list of indicators compared
    selection_df = pd.DataFrame(indic_selection)
    selection_df = selection_df.sort_values('rvalue_abs', ascending=False)

    selection_df = selection_df.reset_index()    
    
    return selection_df.head(50)


# Perform linear regression on full data

In [9]:
# run the function linear regression and compare all indicators by year

Linear_regression_fulldata = select_indicator(full_indicators_countries_df,3)
# synthesis_df.sort_values('pvalue', ascending=False)
Linear_regression_fulldata

,index,indicator A,indicator B,slope,intercept,rvalue,pvalue,std,rvalue_abs
0,21,immigration_100k,government consumption exp (% of GDP),-0.006305,1.453235e+01,-0.263734,0.000181,0.001651,0.263734
1,24,immigration_100k,"Industry (including construction), value added...",-0.006768,2.446454e+01,-0.188676,0.007459,0.002504,0.188676
2,10,immigration_100k,Current account balance (% of GDP),-0.003046,-1.736276e+00,-0.182965,0.012674,0.001210,0.182965
3,28,immigration_100k,Military expenditure (% of GDP),-0.000564,1.269568e+00,-0.167182,0.019176,0.000239,0.167182
4,30,immigration_100k,Population growth (annual %),-0.000636,1.209300e+00,-0.165634,0.018484,0.000268,0.165634
5,29,immigration_100k,Population density,0.054948,1.076233e+02,0.163487,0.026590,0.024578,0.163487
6,31,immigration_100k,% of total population below 5 meters,0.002174,3.588594e+00,0.163233,0.026004,0.000969,0.163233
7,45,immigration_100k,"Unemployment, total (% of total labor force) (...",0.002881,5.660942e+00,0.152213,0.034583,0.001353,0.152213
8,16,immigration_100k,% Firms using banks,-0.004548,8.517041e+00,-0.141418,0.046886,0.002274,0.141418
9,36,immigration_100k,Poverty $6.85 a day (% of population),-0.011799,2.196225e+01,-0.131994,0.060486,0.006250,0.131994


# Perform regression on Annual indicators data

In [10]:
### import and read csv file

path = 'Output/full_indicators_by_country_by_year.csv'
full_indicators_countries_year_df = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

full_indicators_countries_year_df.head(5)

,index,country,iso3Code,year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)",...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,0,Afghanistan,AFG,2015,27.6,71.5,58.123668,3.724667,327.0,38.803,...,0.009,0.292137,3.299608,0.00,0.000,0.000,9.052,20.0,3.294,7.758981
1,1,Afghanistan,AFG,2016,28.8,97.7,58.123668,3.635915,327.0,37.936,...,0.013,0.324903,4.333403,0.00,0.000,0.000,10.133,20.0,3.294,7.651162
2,2,Afghanistan,AFG,2017,30.3,97.7,58.123668,3.501771,327.0,37.342,...,0.017,0.333475,4.353572,15.46,12.283,16.155,11.184,20.0,3.294,9.694882
3,3,Afghanistan,AFG,2018,31.4,93.4,58.276988,2.961201,327.0,36.927,...,0.016,0.344999,3.323800,15.46,12.283,16.155,11.196,20.0,3.294,9.688910
4,4,Afghanistan,AFG,2019,32.6,97.7,58.276988,3.023536,327.0,36.466,...,0.016,0.291267,2.974238,15.46,12.283,16.155,11.185,20.0,3.294,10.275749


In [11]:
# run the function linear regression and compare all indicators by year

Linear_regression_fulldata_year = select_indicator(full_indicators_countries_year_df,3)
# synthesis_df.sort_values('pvalue', ascending=False)
Linear_regression_fulldata_year

,index,indicator A,indicator B,slope,intercept,rvalue,pvalue,std,rvalue_abs
0,22,immigration_100k,government consumption exp (% of GDP),-0.059356,1.492849e+01,-0.282028,3.079109e-31,0.005000,0.282028
1,25,immigration_100k,"Industry (including construction), value added...",-0.064049,2.478084e+01,-0.221990,2.219404e-19,0.007024,0.221990
2,32,immigration_100k,% of total population below 5 meters,0.022581,3.006315e+00,0.217722,9.345048e-18,0.002599,0.217722
3,29,immigration_100k,Military expenditure (% of GDP),-0.006048,1.320856e+00,-0.213247,1.237412e-17,0.000699,0.213247
4,30,immigration_100k,Population density,0.571375,1.007920e+02,0.196144,1.301345e-14,0.073412,0.196144
5,31,immigration_100k,Population growth (annual %),-0.006220,1.263138e+00,-0.168658,6.233488e-12,0.000898,0.168658
6,34,immigration_100k,"Population, total",-71458.776796,1.505780e+07,-0.156173,1.322853e-09,11708.317463,0.156173
7,4,immigration_100k,Armed forces (% labor force),-0.003012,7.986952e-01,-0.151173,3.394282e-09,0.000507,0.151173
8,33,immigration_100k,Population living in slums (% of urban populat...,-0.065953,1.231282e+01,-0.128734,3.279021e-07,0.012859,0.128734
9,35,immigration_100k,Poverty $2.15 a day (% of population),-0.002878,5.328798e-01,-0.109116,5.735474e-05,0.000713,0.109116


# Perform Linear regression on clustered data

In [12]:
### import and read csv file

path = 'Output_analysis/Clusters_analysis.csv'
clusters = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

clusters.head(5)

,country,countries_cluster_original_data,countries_cluster_withPCA
0,Iceland,0,0
1,Latvia,0,0
2,Germany,0,0
3,Russian Federation,0,0
4,Romania,0,0


In [13]:
new_df = pd.merge(full_indicators_countries_df, clusters, on = 'country' )
new_df

,index,country,iso3Code,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",...,"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data,countries_cluster_withPCA
0,0,Afghanistan,AFG,32.400000,91.555556,58.432353,2.681272,327.0,36.628111,0.000000,...,3.220058,10.011889,7.579333,9.741222,11.616111,20.0,3.294,218.749241,2,1
1,1,Albania,ALB,81.455556,99.977778,42.382186,0.614119,1485.0,10.610444,78.238375,...,17.127848,13.158667,10.648444,14.884000,12.600667,37.0,1.809,197.990096,3,3
2,2,Algeria,DZA,99.677778,99.655556,17.353871,2.628728,89.0,23.088556,0.000000,...,0.736472,13.805556,8.501111,8.757778,12.170889,36.0,2.110,94.177289,3,3
3,3,American Samoa,ASM,0.000000,0.000000,14.150000,0.000000,0.0,11.422222,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0,2
4,4,Andorra,AND,100.000000,100.000000,39.983452,0.000000,0.0,6.466667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,198,Viet Nam,VNM,91.644444,99.744444,39.126963,0.943333,1821.0,15.627444,0.000000,...,5.989108,3.482444,1.287222,2.345889,1.785333,41.0,1.802,43.300050,0,3
199,199,West Bank and Gaza,PSE,0.000000,99.977778,68.829501,0.000000,402.0,29.736444,0.000000,...,0.000000,30.662889,22.519000,24.193444,25.038000,0.0,2.872,128.246720,3,3
200,200,"Yemen, Rep.",YEM,52.588889,72.344444,44.401176,0.503899,167.0,31.745667,0.000000,...,15.049805,0.000000,0.000000,0.000000,17.822111,16.0,3.397,17.149918,2,2
201,201,Zambia,ZMB,11.211111,41.877778,32.038066,0.266775,1020.0,35.655000,65.282245,...,14.786883,6.638556,6.436556,9.354556,5.490778,37.0,1.948,3.345682,1,1


In [14]:
# to have the population of each cluster
x = new_df.groupby('countries_cluster_withPCA')['country'].count()
x

countries_cluster_withPCA
0    55
1    44
2    41
3    63
Name: country, dtype: int64

In [16]:
# create empty list to store the results
results_list = []

# loop for each cluster and generate a linear regression DF
for i in range(0,4) :
    
    result_dict = {}
    
    cluster_num = i

    # filter the DF regarding the cluster

    clustered_df = new_df[new_df['countries_cluster_withPCA']==cluster_num]
    
    # prepare the DF to the linear regression
    clustered_df = clustered_df.drop(columns=['countries_cluster_original_data', 'countries_cluster_withPCA'])
    
    # run the linear regression
    Linear_regression_with_PCA = select_indicator(clustered_df,3)

    # export the reslt of the linear regression to EXCEL file
    file_destination = f'Output_analysis/Linear_regression_with_PCA_cluster_number_{cluster_num}.xlsx'
    Linear_regression_with_PCA.to_excel(file_destination)

    # extract the max rvalue
    rvalue_max = Linear_regression_with_PCA['rvalue_abs'].max()

    # store the max rvalue in a dict to be appended in the list results
    result_dict = {
            'cluster' : i ,
            'rvalue' : rvalue_max
                }
    results_list.append(result_dict)

# create DF with list of results
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('rvalue', ascending=False)
results_df

,cluster,rvalue
3,3,0.438036
1,1,0.401879
2,2,0.377876
0,0,0.366653


In [17]:
# filter the DF regarding the cluster

## choose the cluster
cluster_num = 3

## filter
clustered_df = new_df[new_df['countries_cluster_withPCA']==cluster_num]
clustered_df = clustered_df.drop(columns=['countries_cluster_original_data', 'countries_cluster_withPCA'])
clustered_df.head()

,index,country,iso3Code,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
1,1,Albania,ALB,81.455556,99.977778,42.382186,0.614119,1485.0,10.610444,78.238375,...,4.822222,5.576404,17.127848,13.158667,10.648444,14.884000,12.600667,37.0,1.809,197.990096
2,2,Algeria,DZA,99.677778,99.655556,17.353871,2.628728,89.0,23.088556,0.000000,...,0.574444,0.162233,0.736472,13.805556,8.501111,8.757778,12.170889,36.0,2.110,94.177289
7,7,Argentina,ARG,99.844444,99.955556,43.163578,0.530716,591.0,15.106778,0.000000,...,8.198889,6.645575,41.525570,2.702000,8.230889,7.775444,8.468333,37.0,1.855,7.117405
8,8,Armenia,ARM,97.800000,99.933333,58.698513,3.256943,562.0,12.983778,29.623738,...,4.643333,13.312760,30.398428,11.418333,10.545778,12.555222,16.791111,47.0,2.052,69.704423
11,11,Azerbaijan,AZE,97.922222,100.000000,57.805163,1.637356,447.0,13.777778,8.073031,...,2.043333,4.232180,8.248160,5.175556,11.237778,4.988889,5.491778,23.0,2.248,25.319819


In [18]:
# run the function linear regression and compare all indicators by year

Linear_regression_with_PCA = select_indicator(clustered_df,3)
# synthesis_df.sort_values('pvalue', ascending=False)
Linear_regression_with_PCA

,index,indicator A,indicator B,slope,intercept,rvalue,pvalue,std,rvalue_abs
0,8,immigration_100k,Control of Corruption: Estimate,6.933416e-04,-5.606968e-01,0.438036,0.000584,0.000190,0.438036
1,46,immigration_100k,Corruption index 2023,8.747384e-03,3.663190e+01,0.401313,0.001632,0.002644,0.401313
2,10,immigration_100k,Current account balance (% of GDP),-3.206583e-03,-3.384274e+00,-0.348814,0.007832,0.001162,0.348814
3,24,immigration_100k,"Industry (including construction), value added...",-6.415302e-03,2.668565e+01,-0.340085,0.006842,0.002290,0.340085
4,12,immigration_100k,"Death rate, crude (per 1,000 people)",1.285902e-03,6.781707e+00,0.318885,0.016602,0.000520,0.318885
5,3,immigration_100k,Armed forces (% labor force),-5.619850e-04,1.201524e+00,-0.307800,0.019848,0.000234,0.307800
6,16,immigration_100k,% Firms using banks,-8.234556e-03,1.508379e+01,-0.292025,0.020215,0.003453,0.292025
7,28,immigration_100k,Military expenditure (% of GDP),-5.875465e-04,1.376096e+00,-0.280765,0.032774,0.000268,0.280765
8,30,immigration_100k,Population growth (annual %),-5.648799e-04,8.548772e-01,-0.258885,0.042178,0.000272,0.258885
9,35,immigration_100k,Poverty $3.65 a day (% of population),-2.701308e-03,4.985088e+00,-0.252416,0.055930,0.001384,0.252416
